In [ ]:
# final kidney prediction

In [1]:
# try model with embedding data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from pathlib import Path
import warnings

In [2]:
# load all data
file_path = 'h_sapiens_abundance_PPI_seq_embeddings.parquet'
df = pd.read_parquet(file_path)

In [11]:
df_selected = df[df['sample_organ'].isin({'BRAIN',
 'CEREBRAL_CORTEX',
 'HEART',
 'LIVER',
 'LUNG',
 'PANCREAS',
 'PLASMA',
 'PLATELET',
 'SPLEEN',
 'WHOLE_ORGANISM',
 'KIDNEY'})]

In [12]:
df_selected = df_selected[df_selected['is_integrated'] == True]

In [13]:
# Prepare the data for regression
test_organ_abundance_human = df_selected[df_selected['sample_organ'] == 'KIDNEY']
train_organs_abundance_human = df_selected[(df_selected['sample_organ'] != 'KIDNEY')]

In [14]:
train_organs_abundance_human

,dataset_id,dataset_name,dataset_description,organism_id,organism_name,sample_organ,is_integrated,quality_score,coverage,publication_link,...,ProteinName,GeneSymbol,Sequence,uniprot_status,nog_id,abundance,EnsemblTranscript,EnsemblGene,PPI_embeddings,sequence_embeddings
56182,3692721588,H.sapiens - Lung (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,LUNG,True,30.3,67.0,N/A,...,Albumin,"['ALB', 'GIG20', 'GIG42', 'PRO0903', 'PRO1708'...",MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,available,NOG009613,27123.000,ENST00000295897,ENSG00000163631,"[0.01572, 0.00785, 0.010025, -0.01097, -0.0174...","[-0.06354, 0.0074, 0.08466, 0.02298, -0.01147,..."
56183,3692721588,H.sapiens - Lung (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,LUNG,True,30.3,67.0,N/A,...,None,['HBA1'],None,None,NOG039241,15420.000,ENST00000320868,ENSG00000206172,"[0.146, -0.04395, -0.00908, 0.03014, -0.02135,...","[0.06995, 0.01424, -0.000669, -0.03812, -0.030..."
56184,3692721588,H.sapiens - Lung (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,LUNG,True,30.3,67.0,N/A,...,Hemoglobin subunit alpha (Alpha-globin) (Hemog...,"['HBA1;', 'HBA2']",MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPH...,available,NOG039241,14856.000,ENST00000251595,ENSG00000188536,"[0.1547, -0.02129, -0.01663, 0.02698, -0.02042...","[0.06995, 0.01424, -0.000669, -0.03812, -0.030..."
56185,3692721588,H.sapiens - Lung (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,LUNG,True,30.3,67.0,N/A,...,"Fatty acid-binding protein, liver (Fatty acid-...","['FABP1', 'FABPL']",MSFSGKYQLQSQENFEAFMKAIGLPEELIQKGKDIKGVSEIVQNGK...,available,NOG009902,7512.000,ENST00000295834,ENSG00000163586,"[0.00449, 0.05606, 0.04944, 0.002422, -0.01275...","[0.01868, -0.0119, 0.00871, -0.02475, 0.003374..."
56186,3692721588,H.sapiens - Lung (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,LUNG,True,30.3,67.0,N/A,...,Vimentin,['VIM'],MSTRSVSSSSYRRMFGGPGTASRPSSSRSYVTTSTRTYSLGSALRP...,available,NOG009158,5633.000,ENST00000544301,ENSG00000026025,"[0.041, -0.0222, 0.04755, -0.0373, -0.003492, ...","[-0.0336, 0.0272, -0.02344, -0.01247, 0.02335,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537093,4290589278,H.sapiens - Cerebral cortex (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,CEREBRAL_CORTEX,True,27.7,60.0,N/A,...,Probable E3 ubiquitin-protein ligase MARCHF10 ...,"['MARCHF10', 'MARCH10', 'RNF190']",MLHDARDRQKFFSDVQYLRDMQHKVDSEYQACLRRQEYRRDPNEKK...,available,NOG031048,0.014,ENST00000583600,ENSG00000173838,"[-0.01678, 0.0817, 0.036, 0.0003529, 0.02655, ...","[0.001326, -0.04422, -0.0074, -0.00086, -0.004..."
1537094,4290589278,H.sapiens - Cerebral cortex (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,CEREBRAL_CORTEX,True,27.7,60.0,N/A,...,Pleckstrin homology-like domain family B member 3,['PHLDB3'],MGTRSSPEEGTPPPLVPECDVEVQPQGHPEESREQEASEVLAEPSS...,available,NOG009476,0.013,ENST00000292140,ENSG00000176531,"[0.02023, -0.05698, 0.001915, -0.00793, -0.005...","[-0.02107, -0.01714, 0.02367, -0.0282, -0.0243..."
1537095,4290589278,H.sapiens - Cerebral cortex (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,CEREBRAL_CORTEX,True,27.7,60.0,N/A,...,Zinc finger MYM-type protein 6 (Transposon-der...,"['ZMYM6', 'Buster2', 'KIAA1353', 'ZNF258']",MKEPLDGECGKAVVPQQELLDKIKEEPDNAQEYGCVQQPKTQESKL...,available,NOG008919,0.013,ENST00000357182,ENSG00000163867,"[0.01541, -0.02754, -0.01037, 0.0035, -0.01964...","[-0.01126, 0.02388, 0.02231, 0.01682, -0.00503..."
1537096,4290589278,H.sapiens - Cerebral cortex (Integrated),integrated dataset: weighted average of all H....,9606,H.sapiens,CEREBRAL_CORTEX,True,27.7,60.0,N/A,...,Pecanex-like protein 2 (Pecanex homolog protei...,"['PCNX2', 'KIAA0435', 'PCNXL2']",MVSQVLQLLRQGVWAALTGGWYHDPEQSKFTNSCHLYLWLFLLLLP...,available,N

In [15]:
test_organ_abundance_human = test_organ_abundance_human[['sample_organ', 'abundance', 'UniprotAccession', 'PPI_embeddings', 'sequence_embeddings']]
train_organs_abundance_human = train_organs_abundance_human[['sample_organ', 'abundance', 'UniprotAccession', 'PPI_embeddings', 'sequence_embeddings']]

# only keep rows with nog_id in kidney_abundance
test_organ_abundance_human = test_organ_abundance_human[test_organ_abundance_human['UniprotAccession'].notna()]
test_organ_abundance_human = test_organ_abundance_human[test_organ_abundance_human['abundance'].notna()]
proteins_kidney = test_organ_abundance_human['UniprotAccession'].unique()

# only keep rows with nog_id in other_abundance
train_organs_abundance_human = train_organs_abundance_human[train_organs_abundance_human['UniprotAccession'].isin(proteins_kidney)]

test_organ_abundance_human = test_organ_abundance_human[test_organ_abundance_human['UniprotAccession'].isin(train_organs_abundance_human['UniprotAccession'])]

# remove duplicates in kidney_abundance_mouse
test_organ_abundance_human = test_organ_abundance_human.drop_duplicates(subset=['UniprotAccession'])

# remove duplicates in other_abundance_mouse per organ
train_organs_abundance_human = train_organs_abundance_human.drop_duplicates(subset=['UniprotAccession', 'sample_organ'])

In [19]:
# split the PPI and sequence embeddings into separate columns by splitting the vector 
def split_embeddings(df, column_name):
    embeddings = df[column_name]
    # Find the embedding length (skip NaN)
    first_valid = embeddings[embeddings.apply(lambda x: isinstance(x, (list, np.ndarray)))].iloc[0]
    embedding_length = len(first_valid)

    # Replace NaN with zeros
    embeddings_filled = embeddings.apply(
        lambda x: x if isinstance(x, (list, np.ndarray)) else [0.0] * embedding_length
    )

    split_df = pd.DataFrame(embeddings.tolist(), columns=[f'{column_name}_{i}' for i in range(embeddings.iloc[0].shape[0])])
    return pd.concat([df.reset_index(drop=True), split_df], axis=1)
train_organs_abundance_human = split_embeddings(train_organs_abundance_human, 'PPI_embeddings')
train_organs_abundance_human = split_embeddings(train_organs_abundance_human, 'sequence_embeddings')

In [20]:
test_expression = test_organ_abundance_human[['sample_organ', 'abundance', 'UniprotAccession']]
train_expression = train_organs_abundance_human[['sample_organ', 'abundance', 'UniprotAccession']]

# log transform the abundance values
test_expression['abundance'] = np.log1p(test_expression['abundance'])
train_expression['abundance'] = np.log1p(train_expression['abundance'])

train_expression_wide = train_expression.pivot(index='UniprotAccession', columns='sample_organ', values='abundance').reset_index()

# recombine the PPI and sequence embeddings with the expression data
# To avoid duplicate rows, drop duplicates by 'UniprotAccession' before merging
train_organs_abundance_human_unique = train_organs_abundance_human.drop_duplicates(subset=['UniprotAccession'])
train_organs_abundance_human_ready = pd.merge(
    train_expression_wide,
    train_organs_abundance_human_unique.drop(columns=['sample_organ', 'abundance', 'PPI_embeddings', 'sequence_embeddings']),
    left_on='UniprotAccession',
    right_on='UniprotAccession',
    how='left'
)

C:\Users\ellen\AppData\Local\Temp\ipykernel_18612\37929950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_expression['abundance'] = np.log1p(test_expression['abundance'])
C:\Users\ellen\AppData\Local\Temp\ipykernel_18612\37929950.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_expression['abundance'] = np.log1p(train_expression['abundance'])


In [22]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr

X = train_organs_abundance_human_ready.fillna(0).set_index('UniprotAccession')
y = test_expression.set_index('UniprotAccession')['abundance']
X = X.loc[y.index]

# Train model without interaction terms
model = XGBRegressor(tree_method='hist', n_jobs=-1, random_state=42,n_estimators=300,
    learning_rate=0.1,
    max_depth=5, reg_alpha=0.5, reg_lambda=2)
model.fit(X, y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [83]:
# Feature importances
importances = model.feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importances_df

,Feature,Importance
3,LIVER,0.123638
4,LUNG,0.033655
2,HEART,0.019061
5,PANCREAS,0.009612
1349,sequence_embeddings_827,0.008019
...,...,...
785,sequence_embeddings_263,0.000000
866,sequence_embeddings_344,0.000000
694,sequence_embeddings_172,0.000000
1204,sequence_embeddings_682,0.000000


In [23]:
# load all data
file_path = 'm_musculus_abundance_PPI_seq_embeddings.parquet'
df_mouse = pd.read_parquet(file_path)

In [60]:
df_mouse_selected = df_mouse[df_mouse['sample_organ'].isin({'BRAIN',
 'CEREBRAL_CORTEX',
 'HEART',
 'LIVER',
 'LUNG',
 'PANCREAS',
 'PLASMA',
 'PLATELET',
 'SPLEEN',
 'WHOLE_ORGANISM'})]

In [61]:
df_mouse_selected = df_mouse_selected[df_mouse_selected['is_integrated'] == True]

In [62]:
# Prepare the data for regression
train_organs_abundance_mouse = df_mouse_selected

In [63]:
genes_to_predict = pd.read_csv('protein-abundance-PMDA2025/test_masked_integrated_rows_M.musculus_KIDNEY.csv.gz', compression='gzip')

In [64]:
train_organs_abundance_mouse = train_organs_abundance_mouse[['sample_organ', 'abundance', 'UniprotAccession', 'PPI_embeddings', 'sequence_embeddings']]

train_organs_abundance_mouse = train_organs_abundance_mouse[train_organs_abundance_mouse['UniprotAccession'].isin(genes_to_predict['UniprotAccession'])]

# remove duplicates in other_abundance_mouse per organ
train_organs_abundance_mouse = train_organs_abundance_mouse.drop_duplicates(subset=['UniprotAccession', 'sample_organ'])

In [65]:
# split the PPI and sequence embeddings into separate columns by splitting the vector 
def split_embeddings(df, column_name):
    embeddings = df[column_name]
    # Find the embedding length (skip NaN)
    first_valid = embeddings[embeddings.apply(lambda x: isinstance(x, (list, np.ndarray)))].iloc[0]
    embedding_length = len(first_valid)

    # Replace NaN with zeros
    embeddings_filled = embeddings.apply(
        lambda x: x if isinstance(x, (list, np.ndarray)) else [0.0] * embedding_length
    )

    split_df = pd.DataFrame(embeddings.tolist(), columns=[f'{column_name}_{i}' for i in range(embeddings.iloc[0].shape[0])])
    return pd.concat([df.reset_index(drop=True), split_df], axis=1)
train_organs_abundance_mouse = split_embeddings(train_organs_abundance_mouse, 'PPI_embeddings')
train_organs_abundance_mouse = split_embeddings(train_organs_abundance_mouse, 'sequence_embeddings')

In [66]:
train_expression_mouse = train_organs_abundance_mouse[['sample_organ', 'abundance', 'UniprotAccession']]

# log transform the abundance values
train_expression_mouse['abundance'] = np.log1p(train_expression_mouse['abundance'])

train_expression_mouse_wide = train_expression_mouse.pivot(index='UniprotAccession', columns='sample_organ', values='abundance').reset_index()

# recombine the PPI and sequence embeddings with the expression data
# To avoid duplicate rows, drop duplicates by 'UniprotAccession' before merging
train_organs_abundance_mouse_unique = train_organs_abundance_mouse.drop_duplicates(subset=['UniprotAccession'])
train_organs_abundance_mouse_ready = pd.merge(
    train_expression_mouse_wide,
    train_organs_abundance_mouse_unique.drop(columns=['sample_organ', 'abundance', 'PPI_embeddings', 'sequence_embeddings']),
    left_on='UniprotAccession',
    right_on='UniprotAccession',
    how='left'
)

C:\Users\ellen\AppData\Local\Temp\ipykernel_18612\1458885912.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_expression_mouse['abundance'] = np.log1p(train_expression_mouse['abundance'])


In [68]:
X = train_organs_abundance_mouse_ready.fillna(0).set_index('UniprotAccession')

# Predict and evaluate
y_pred = np.expm1(model.predict(X))

In [79]:
# put y_pred in a df including protein names
prediction_df = pd.DataFrame(y_pred, index=X.index, columns=['abundance'])

In [80]:
# add missing genes from genes_to_predict 
missing_genes = genes_to_predict['UniprotAccession'][~genes_to_predict['UniprotAccession'].isin(prediction_df.index)]

# Create a DataFrame for missing genes with NaN abundance
missing_df = pd.DataFrame({'abundance': 0}, index=missing_genes)
missing_df.index.name = 'UniprotAccession'

# Concatenate predictions with missing genes
prediction_df = pd.concat([prediction_df, missing_df])


In [81]:
prediction_df

,abundance
UniprotAccession,
A0A087WPY5,2.556004
A0A087WQ89,0.504605
A0A087WR13,0.002975
A0A0G2JDV3,0.298896
A0A0N4SUH6,1.573978
...,...
Q8BTG7,0.000000
E9Q2M9,0.000000
Q8BQ48,0.000000


In [82]:
# save as csv to team 1 github
prediction_df.to_csv('kidney_predictions.csv')
